# Problem 7
Let $A$ be the $20000 \times 20000$ matrix whose entries are 0 everywhere except for the primes 2,3,5,7,...,224737 along the main diagonal and the number 1 in all the positions $ a_{ij} $ with $ |i-j| = 1,2,4,8,\cdots,16384$. What is the (1,1) entry of $A^{-1}$

### Some introductory code 

In [1]:
import numpy as np

def is_prime(N):
    return N > 1 and all([N%i != 0 for i in range(2, int(N**0.5+1))])
primes = [i for i in range(2,250000) if is_prime(i)][:20000]
entries = [(i,i,primes[i]) for i in range(len(primes))]
for diff in [2**j for j in range(15)]:
    for i in range(20000):
        j1, j2 = i+diff, i-diff
        if 0<=j1<20000: entries.append((i,j1,1))
        if 0<=j2<20000: entries.append((i,j2,1))
entries = np.array(entries)
print(entries.shape)

(554466, 3)


### Attempt 1: Using standard libraries

We avoid calculating $A^{-1}$ explicitly. Instead, note that if we have $\hat{v}$ be the first column of $ A^{-1} $, then we get that $ A \hat{v} = (1,0,\cdots, 0)^{T}$. Let's solve this system. 

In [2]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

row, col, data = entries[:, 0], entries[:, 1], entries[:, 2]

A = coo_matrix((data, (row, col))).tocsc()
b = np.zeros(A.shape[0])
b[0] = 1
# v = spsolve(A,b)
# Timeout

The above code times out - so instead, let's try to use an iterative method to solve this linear system.

In [11]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import bicg
row = np.array([i[0] for i in entries])
col = np.array([i[1] for i in entries])
data = np.array([i[2] for i in entries])

A = coo_matrix((data, (row, col))).tocsc()
b = np.zeros(A.shape[0])
b[0] = 1

iterations = 0
def callback(xk):
    global iterations
    iterations += 1
    if iterations % 10 == 0: print("Iteration {}: current val = {}".format(iterations, xk[0]))
x, exit_code = bicg(A,b,tol=1e-50,callback=callback)
print(x[0])

Iteration 10: current val = 0.5160474805069852
Iteration 20: current val = 0.5424188205227113
Iteration 30: current val = 0.5649984704641765
Iteration 40: current val = 0.5809367071267324
Iteration 50: current val = 0.6096730237142656
Iteration 60: current val = 0.6190393486130085
Iteration 70: current val = 0.6263245512238234
Iteration 80: current val = 0.6508621776392641
Iteration 90: current val = 0.6577928020000656
Iteration 100: current val = 0.6608853920165397
Iteration 110: current val = 0.6646370853741161
Iteration 120: current val = 0.6769767188475972
Iteration 130: current val = 0.686598661153931
Iteration 140: current val = 0.6950283972972651
Iteration 150: current val = 0.6985776576929462
Iteration 160: current val = 0.700549226696957
Iteration 170: current val = 0.7016993660417654
Iteration 180: current val = 0.7037530670049756
Iteration 190: current val = 0.70578202448334
Iteration 200: current val = 0.7080325880243894
Iteration 210: current val = 0.7107390862052434
Itera

In [4]:
def is_prime(N):
    return N > 1 and all([N%i != 0 for i in range(2, int(N**0.5+1))])

[i for i in range(100) if is_prime(i)]

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

(554466, 3)

In [7]:
A

<20000x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 554466 stored elements in Compressed Sparse Column format>

In [10]:
import scipy.sparse.linalg as linalg
help(linalg)

Help on package scipy.sparse.linalg in scipy.sparse:

NAME
    scipy.sparse.linalg

DESCRIPTION
    Sparse linear algebra (:mod:`scipy.sparse.linalg`)
    
    .. currentmodule:: scipy.sparse.linalg
    
    Abstract linear operators
    -------------------------
    
    .. autosummary::
       :toctree: generated/
    
       LinearOperator -- abstract representation of a linear operator
       aslinearoperator -- convert an object to an abstract linear operator
    
    Matrix Operations
    -----------------
    
    .. autosummary::
       :toctree: generated/
    
       inv -- compute the sparse matrix inverse
       expm -- compute the sparse matrix exponential
       expm_multiply -- compute the product of a matrix exponential and a matrix
    
    Matrix norms
    ------------
    
    .. autosummary::
       :toctree: generated/
    
       norm -- Norm of a sparse matrix
       onenormest -- Estimate the 1-norm of a sparse matrix
    
    Solving linear problems
    -------

In [ ]:
import matplotlib.pyplot as plt

X, Y = [], []

iterations = 0
def callback(xk):
    global iterations, X, Y
    iterations += 1
    X.append(iterations); Y.append(xk[0])
    if iterations % 10 == 0: print("Iteration {}: current val = {}".format(iterations, xk[0]))

linalg.qmr(A, b, callback=callback)

plt.plot(X, Y)
plt.show()

Iteration 10: current val = 0.19423089190652099
Iteration 20: current val = 0.23417992806110038
Iteration 30: current val = 0.2656958595038305
Iteration 40: current val = 0.3054222707982813
Iteration 50: current val = 0.3135587933679594
Iteration 60: current val = 0.3556327670184943
Iteration 70: current val = 0.3789918731142482
Iteration 80: current val = 0.38544876096001984
Iteration 90: current val = 0.4121182251863883
Iteration 100: current val = 0.44925068976200094
Iteration 110: current val = 0.475242867281242
Iteration 120: current val = 0.4816244852405636
Iteration 130: current val = 0.4902349361021055
Iteration 140: current val = 0.49962816500256785
Iteration 150: current val = 0.5174883308803532
Iteration 160: current val = 0.5488219155722759
Iteration 170: current val = 0.5773560934837708
Iteration 180: current val = 0.5899200677253085
Iteration 190: current val = 0.6001837437958351
Iteration 200: current val = 0.6084965638571498
Iteration 210: current val = 0.61447098156912